In [2]:
import arcpy
import os
from arcpy.sa import *

# 数据集融合

In [13]:
# 设置包含POI文件的文件夹路径
folder_path = r"E:\stu1\UFZs\poi_data\2015"
# 设置工作环境
arcpy.env.workspace = folder_path

In [14]:
# 查找并列出文件夹中所有的POI文件
poi_files = arcpy.ListFeatureClasses()
# 合并后的输出文件名
merged_poi_output = r"E:\stu1\Projects\POI2023\POIs.gdb\POI15"

In [15]:
# 合并POI数据
arcpy.management.Merge(inputs = poi_files, output = merged_poi_output)

<Result 'E:\\stu1\\Projects\\POI2023\\POIs.gdb\\POI15'>

In [16]:
folder_path = r"E:\stu1\UFZs\poi_data\2016"
arcpy.env.workspace = folder_path
poi_files = arcpy.ListFeatureClasses()
merged_poi_output = r"E:\stu1\Projects\POI2023\POIs.gdb\POI16"
arcpy.management.Merge(inputs = poi_files, output = merged_poi_output)

<Result 'E:\\stu1\\Projects\\POI2023\\POIs.gdb\\POI16'>

In [17]:
# 基本文件夹路径
base_folder_path = r"E:\stu1\UFZs\poi_data"
output_base_path = r"E:\stu1\Projects\POI2023\POIs.gdb"

# 遍历2017年到2023年的数据
for year in range(2017, 2024):
    # 设置当前年份的工作空间
    folder_path = f"{base_folder_path}\{year}"
    arcpy.env.workspace = folder_path
    
    # 列出当前工作空间中的要素类
    poi_files = arcpy.ListFeatureClasses()
    
    # 设置融合后的输出路径
    merged_poi_output = f"{output_base_path}\POI{year}"
    
    # 执行融合操作
    arcpy.management.Merge(inputs=poi_files, output=merged_poi_output)

    print(f"完成{year}年的POI数据融合")


完成2017年的POI数据融合
完成2018年的POI数据融合
完成2019年的POI数据融合
完成2020年的POI数据融合
完成2021年的POI数据融合
完成2022年的POI数据融合
完成2023年的POI数据融合


# KDE

In [3]:
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POIs.gdb"

## KDE

poi_data = r"E:\stu1\Projects\POI2023\POIs.gdb\POI12"
kde_output = r"E:\stu1\Projects\POI2023\POIs.gdb\kde\kde12"
kde = KernelDensity(poi_data, "NONE")
kde.save(kde_output)

In [7]:
# 基础路径
base_path = r"E:\stu1\Projects\POI2023\POIs.gdb"

# 遍历2012年到2023年
for year in range(12, 24):
    # POI数据和KDE输出路径
    poi_data = f"{base_path}\POI{year}"
    kde_output = f"{base_path}\kde{year}"
    
    # 执行KDE
    kde = KernelDensity(poi_data, "NONE")
    kde.save(kde_output)
    
    print(f"完成{year}年的KDE分析")

完成12年的KDE分析
完成13年的KDE分析
完成14年的KDE分析
完成15年的KDE分析
完成16年的KDE分析
完成17年的KDE分析
完成18年的KDE分析
完成19年的KDE分析
完成20年的KDE分析
完成21年的KDE分析
完成22年的KDE分析
完成23年的KDE分析


## POI投影

In [2]:
# Define the input WGS 84 spatial reference (EPSG: 4326)
input_sr = arcpy.SpatialReference(4326)
# output UTM Zone 50N 
output_sr = arcpy.SpatialReference(32650)

In [3]:
# 基础路径
base_path = r"E:\stu1\Projects\POI2023\POIs.gdb"

# 遍历2012年到2023年
for year in range(12, 24):
    # KDE数据的输入和输出路径
    input_kde = f"{base_path}\kde{year}"
    projected_kde = f"{base_path}\Pkde{year}"
    
    # Project
    arcpy.management.ProjectRaster(input_kde, projected_kde, output_sr)
    
    print(f"完成{year}年的KDE投影")

完成12年的KDE投影
完成13年的KDE投影
完成14年的KDE投影
完成15年的KDE投影
完成16年的KDE投影
完成17年的KDE投影
完成18年的KDE投影
完成19年的KDE投影
完成20年的KDE投影
完成21年的KDE投影
完成22年的KDE投影
完成23年的KDE投影


## 中值二分遍历

In [2]:
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POIs.gdb"

In [3]:
projected_kde = r"E:\stu1\Projects\POI2023\POIs.gdb\Pkde12"

In [4]:
first = "first12"
second = "second12"
third = "third12"

In [6]:
# 获取Projected_kde的平均值
kde_stats = arcpy.GetRasterProperties_management(projected_kde, "MEAN")
threshold_value = float(kde_stats.getOutput(0))

In [16]:
binary_first = Raster(projected_kde) > threshold_value
# 保存栅格数据到Geodatabase
binary_first.save(first)

In [7]:
# Mask raster to include only values above the initial threshold
masked_raster = Con(Raster(projected_kde) > threshold_value, Raster(projected_kde))

# Calculate the mean of the masked raster
mean_of_high_values = float(arcpy.GetRasterProperties_management(masked_raster, "MEAN").getOutput(0))

# Create final binary raster using the second mean as threshold
binary_second = Con(masked_raster > mean_of_high_values, 1, 0)

# 保存栅格数据到Geodatabase
binary_second.save(second)

In [8]:
# Mask raster to include only values above the second threshold
masked_raster_second = Con(binary_second == 1, Raster(projected_kde))

# Calculate the mean of the second masked raster
mean_of_higher_values = float(arcpy.GetRasterProperties_management(masked_raster_second, "MEAN").getOutput(0))

# Create the final binary raster using the third mean as threshold
binary_third = Con(Raster(projected_kde) > mean_of_higher_values, 1, 0)

# 保存栅格数据到Geodatabase
binary_third.save(third)

对13年到23年的数据循环语句自动化实现

In [9]:
# 循环处理13年到23年的数据
for year in range(13, 24):
    # 定义每年的Projected KDE路径
    projected_kde = f"E:\\stu1\\Projects\\POI2023\\POIs.gdb\\Pkde{year}"

    # 计算阈值
    kde_stats = arcpy.GetRasterProperties_management(projected_kde, "MEAN")
    threshold_value = float(kde_stats.getOutput(0))

    # 第一个二值栅格
    binary_first = Raster(projected_kde) > threshold_value
    binary_first.save(f"first{year}")

    # 使用初始阈值掩膜栅格
    masked_raster = Con(Raster(projected_kde) > threshold_value, Raster(projected_kde))

    # 计算掩膜栅格的平均值
    mean_of_high_values = float(arcpy.GetRasterProperties_management(masked_raster, "MEAN").getOutput(0))

    # 创建第二个二值栅格
    binary_second = Con(masked_raster > mean_of_high_values, 1, 0)
    binary_second.save(f"second{year}")

    # 使用第二阈值掩膜栅格
    masked_raster_second = Con(binary_second == 1, Raster(projected_kde))

    # 计算第二掩膜栅格的平均值
    mean_of_higher_values = float(arcpy.GetRasterProperties_management(masked_raster_second, "MEAN").getOutput(0))

    # 创建第三个二值栅格
    binary_third = Con(Raster(projected_kde) > mean_of_higher_values, 1, 0)
    binary_third.save(f"third{year}")


# 划定城市化区域

In [4]:
# 循环处理12年到23年的数据
for year in range(12, 24):
    # 定义每年的bianry_raster路径
    binary_raster = f"E:\\stu1\\Projects\\POI2023\\POIs.gdb\\second{year}"
    # 定义output路径
    output_polygon = f"E:\\stu1\\Projects\\POI2023\\POIs.gdb\\polysec{year}"
    
    # 保留值为1的栅格区域
    binary_raster_filtered = Con(Raster(binary_raster) == 1, 1)
    
    # Raster to Polygon
    arcpy.RasterToPolygon_conversion(binary_raster_filtered, output_polygon, "NO_SIMPLIFY")
    
    print(f"完成{year}年的二值栅格矢量化")

完成12年的二值栅格矢量化
完成13年的二值栅格矢量化
完成14年的二值栅格矢量化
完成15年的二值栅格矢量化
完成16年的二值栅格矢量化
完成17年的二值栅格矢量化
完成18年的二值栅格矢量化
完成19年的二值栅格矢量化
完成20年的二值栅格矢量化
完成21年的二值栅格矢量化
完成22年的二值栅格矢量化
完成23年的二值栅格矢量化


In [6]:
binary_raster = r"

In [7]:
# 保留值为1的栅格区域
binary_raster_filtered = Con(Raster(binary_raster) == 1, 1)

In [4]:
# 定义初始输入
poi_data = r"E:\stu1\Projects\POI2023\POI2023.gdb\MergedPOI23"
kde_output = r"E:\stu1\Projects\POI2023\POI2023.gdb\kde_output"
binary_raster = r"E:\stu1\Projects\POI2023\POI2023.gdb\binary_raster"
output_polygon = r"E:\stu1\Projects\POI2023\POI2023.gdb\output_polygon"
convex_hull_output = r"E:\stu1\Projects\POI2023\POI2023.gdb\convex_hull_output"

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\projected_kde'>

## BINARY RASTER

In [5]:
# 保留值为1的栅格区域
binary_raster_filtered = Con(Raster(binary_raster) == 1, 1)

In [6]:
# Raster to Polygon
arcpy.RasterToPolygon_conversion(binary_raster_filtered, output_polygon, "NO_SIMPLIFY")

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\output_polygon'>

In [ ]:
# 去除过小斑块，选择重点斑块，并将选中的斑块导出

In [7]:
input_polygon = r"E:\stu1\Projects\POI2023\POI2023.gdb\selected_polygon"
dissolved_polygon = r"E:\stu1\Projects\POI2023\POI2023.gdb\dissolved_polygon"

In [8]:
# Dissolve all polygons into a single feature
arcpy.Dissolve_management(input_polygon, dissolved_polygon)

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\dissolved_polygon'>

In [9]:
# Generate a single convex hull for the dissolved feature
arcpy.MinimumBoundingGeometry_management(dissolved_polygon, convex_hull_output, "CONVEX_HULL")

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\convex_hull_output'>

# 裁剪研究区域数据

In [6]:
# Define the paths to your datasets
convex_hull = r"E:\stu1\Projects\POI2023\POI2023.gdb\convex_hull_output"
poi_data = r"E:\stu1\Projects\POI2023\POI2023.gdb\MergedPOI23"
road_network_data = r"E:\stu1\Projects\POI2023\POI2023.gdb\UFZs23\road23"
land_use_data = r"E:\stu1\UFZs\lucc\2020.tif"

# Define the output paths
clipped_poi = r"E:\stu1\Projects\POI2023\POI2023.gdb\clipped_poi"
clipped_road_network = r"E:\stu1\Projects\POI2023\POI2023.gdb\clipped_road"
clipped_land_use = r"E:\stu1\Projects\POI2023\POI2023.gdb\clipped_lucc"

In [7]:
# Clip the datasets
arcpy.Clip_analysis(poi_data, convex_hull, clipped_poi)

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\clipped_poi'>

In [5]:
# Clip the datasets
arcpy.Clip_analysis(road_network_data, convex_hull, clipped_road_network)

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\clipped_road'>

In [10]:
# Clip the raster
# arcpy.Clip_management(land_use_data, "#", clipped_land_use, convex_hull,"#", "NONE")

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\clipped_lucc'>

In [11]:
# Perform the extract by mask operation
masked_raster = ExtractByMask(land_use_data,convex_hull)
masked_raster.save(clipped_land_use)

# 统一空间坐标系

In [13]:
# Output datasets
projected_poi = r"E:\stu1\Projects\POI2023\POI2023.gdb\projected_poi"
projected_road = r"E:\stu1\Projects\POI2023\POI2023.gdb\projected_road"

In [14]:
# UTM Zone 50N spatial reference (EPSG: 32650)
utm50n_sr = arcpy.SpatialReference(32650)

In [15]:
# Reproject datasets
arcpy.Project_management(clipped_poi, projected_poi, utm50n_sr)
arcpy.Project_management(clipped_road_network, projected_road, utm50n_sr)

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\projected_road'>

# 处理2023年POI数据

#### 步骤一：用ring3裁剪23年的POI数据

In [3]:
ring3 = r"E:\stu1\Projects\POI2023\POI2023.gdb\ring3"
poi = r"E:\stu1\Projects\POI2023\POIs.gdb\POI23"
ring3_poi = r"E:\stu1\Projects\POI2023\POIs.gdb\Ring3_POI"

In [4]:
arcpy.Clip_analysis(poi, ring3, ring3_poi)

<Result 'E:\\stu1\\Projects\\POI2023\\POIs.gdb\\Ring3_POI'>

#### 步骤二：分类抽取并存储不同类别的POI

In [2]:
# 输入的POI要素类
input_feature_class = r"E:\stu1\Projects\POI2023\POIs.gdb\Ring3_POI"

In [8]:
arcpy.Delete_management(input_poi_layer)

NameError: name 'input_poi_layer' is not defined

In [10]:
if arcpy.Exists("inputPoiLayer"):  # 检查图层是否存在
    arcpy.management.Delete("inputPoiLayer")  # 如果存在，则删除

# 之后再创建图层
input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, "inputPoiLayer").getOutput(0)


In [11]:
# 定义不同功能区对应的POI类型
typecode_ranges = {
    'R': [str(num) for num in range(120300, 120305)] + # 住宅区
        ["120000"],   # 商务住宅相关
    
    'A': ["080400", "080401", "080402"] + # 度假疗养场所
        ["080600", "080601", "080602", "080603"] + # 影剧院
        ["090100", "090101", "090102"] + # 综合医院
        [f"{num:06d}" for num in range(90200, 90212)] +  # 专科医院
        ["090000", "090400", "090500"] +
        [f"{num:06d}" for num in range(130100, 130108)] +  # 政府机关
        [f"{num:06d}" for num in range(130400, 130410)] +  # 社会团体
        ["130200", "130201", "130202"] + # 外国机构
        ["130700", "130701", "130702", "130703"] + # 工商税务机构
        [f"{num:06d}" for num in range(130600, 130607)] +  # 交通车辆管理
        ["140100", "140101", "140102"] + # 博物馆
        ["140200", "140201"] + # 展览馆
        ["140300", "140400", "140500", "140600", "140800", "140900", "141000"] +
        [f"{num:06d}" for num in range(141100, 141106 )] + # 传媒机构
        [f"{num:06d}" for num in range(141200, 141208 )] + # 学校
        ["141300", "141400"],    # 公共管理与公共服务设施用地
    
    'B': [f"{num:06d}" for num in range(50100, 50124)] +  # 中餐厅
        [f"{num:06d}" for num in range(50200, 50218)] +  # 外国餐厅
        [f"{num:06d}" for num in range(60100, 60104)] +  # 商场
        [f"{num:06d}" for num in range(60400, 60416)] +  # 超级市场
        ["061000", "061001"] + # 特色商业街
        [f"{num:06d}" for num in range(100100, 100106)] +  # 宾馆酒店
        [f"{num:06d}" for num in range(160100, 160153)] +  # 银行
        [f"{num:06d}" for num in range(160400, 160409)] +  # 保险公司
        [f"{num:06d}" for num in range(160500, 160502)] + # 证券公司
        ["120201", "120203"], #楼宇     # 商业服务业设施用地
    
    'M': ["120100"] +  # 产业园区
        ["170300"] + # 工厂
        [f"{num:06d}" for num in range(170200, 170210)],   # 公司   # 工业用地
    
    'S': ["150200"] + # 火车站
        ["150104"] + # 飞机场
        ["150300"] + # 港口相关
        ["180200"] + # 收费站
        ["180300"], # 服务区 # 交通与道路设施用地
    
    'G': [f"{num:06d}" for num in range(110100, 110106)] +  # 公园广场
        [f"{num:06d}" for num in range(110200, 110211)]    # 风景名胜 # 绿地与广场用地
}

In [12]:
for category, codes in typecode_ranges.items():
    # 创建查询
    query = "typecode IN ('{}')".format("', '".join(codes))

    # 按类型代码筛选POI
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)

    # 保存筛选结果为新的要素类
    output_feature_class = "E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_{}".format(category)
    arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)

    print("分类保存完成：", output_feature_class)

    # 清除当前的选择
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

print("所有分类完成")

分类保存完成： E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_R
分类保存完成： E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_A
分类保存完成： E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_B
分类保存完成： E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_M
分类保存完成： E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_S
分类保存完成： E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_G
所有分类完成


#### 步骤三：构建POI的影响因子赋分调整字段

面积权重和公众认知度权重都在0到1之间，然后两者按照5：5的比例，确定最终的影响因子值

##### 居住用地R

In [13]:
shapefile_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_R"

In [14]:
# 添加字段
fields_to_add = [
    ('Area_Rate', 'FLOAT'),
    ('Popularity_Rate', 'FLOAT'),
    ('Weight', 'FLOAT')
]

for field_name, field_type in fields_to_add:
    arcpy.AddField_management(shapefile_R, field_name, field_type)
    
print("完成字段添加 for 类型 R")

完成字段添加 for 类型 R


In [15]:
# 更新面积评分字段

# 定义一个字典，映射 typecode 值到相应的 Area_Rate 评分
typecode_to_arearate = {
    "120300": 1.0,
    "120302": 0.5,
    "120000": 0.5,
    "120301": 0.1,
    "120303": 0.1,
    "120304": 0.1,
}

# 使用 arcpy.da.UpdateCursor 遍历要素类
fields = ['typecode', 'Area_Rate']  # 指定要更新的字段和依据的字段
with arcpy.da.UpdateCursor(shapefile_R, fields) as cursor:
    for row in cursor:
        typecode = row[0]  # 获取当前记录的 typecode 值
        if typecode in typecode_to_arearate:
            row[1] = typecode_to_arearate[typecode]  # 根据映射字典更新 Area_Rate 值
        else:
            row[1] = 0  # 如果 typecode 不在字典中，可以选择赋予一个默认值
        cursor.updateRow(row)  # 更新记录

print("面积评分更新完成 for 类型 R")

面积评分更新完成 for 类型 R


In [16]:
# 更新公众认知度评分字段
with arcpy.da.UpdateCursor(shapefile_R, ["Popularity_Rate"]) as cursor:
    for row in cursor:
        row[0] = 0.1  
        cursor.updateRow(row)

print("公众认知度更新完成 for 类型 R")

公众认知度更新完成 for 类型 R


In [18]:
# 计算权重字段
expression = "(!Area_Rate! + !Popularity_Rate!) / 2"  
arcpy.CalculateField_management(shapefile_R, 'Weight', expression, "PYTHON3")
print("权重字段更新完成 for 类型 R")

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\poi23\\POI_R'>

##### 公服用地A

In [19]:
shapefile_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_A"

In [20]:
# 添加字段
fields_to_add = [
    ('Area_Rate', 'FLOAT'),
    ('Popularity_Rate', 'FLOAT'),
    ('Weight', 'FLOAT')
]

for field_name, field_type in fields_to_add:
    arcpy.AddField_management(shapefile_A, field_name, field_type)
    
print("完成字段添加 for 类型 A")

完成字段添加 for 类型 A


In [21]:
# 更新面积评分字段

# 定义映射，将typecode范围与评分值关联
typecode_to_arearate = {
    tuple([f"{num:06d}" for num in range(130700, 130704)]): 0.01,
    tuple([f"{num:06d}" for num in range(130600, 130607)]): 0.01,
    "141000": 0.1,
    tuple([f"{num:06d}" for num in range(141100, 141106)]): 0.1,
    "141400": 0.1,
    "090000": 0.1,
    "090400": 0.1,
    "090500": 0.1,
    tuple([f"{num:06d}" for num in range(130400, 130410)]): 0.1,
    tuple([f"{num:06d}" for num in range(130200, 130203)]): 0.1,
    tuple([f"{num:06d}" for num in range(140200, 140202)]): 0.3,
    "140300": 0.3,
    "140400": 0.3,
    "140500": 0.3,
    "140600": 0.3,
    "140800": 0.3,
    "140900": 0.3,
    "141300": 0.3,
    tuple([f"{num:06d}" for num in range(80400, 80403)]): 0.3,
    tuple([f"{num:06d}" for num in range(80600, 80604)]): 0.3,
    tuple([f"{num:06d}" for num in range(140100, 140103)]): 0.5,
    tuple([f"{num:06d}" for num in range(90200, 90212)]): 0.8,
    tuple([f"{num:06d}" for num in range(130100, 130108)]): 0.8,
    tuple([f"{num:06d}" for num in range(141200, 141208)]): 1,
    tuple([f"{num:06d}" for num in range(90100, 90103)]): 1,
}


# 使用 arcpy.da.UpdateCursor 更新Area_Rate字段
fields = ['typecode', 'Area_Rate']  # 需要处理的字段
with arcpy.da.UpdateCursor(shapefile_A, fields) as cursor:
    for row in cursor:
        typecode = row[0]
        area_rate_updated = False
        
        # 遍历映射规则
        for key, rate in typecode_to_arearate.items():
            if isinstance(key, tuple):  # 如果键是元组（即范围）
                if typecode in key:
                    row[1] = rate
                    area_rate_updated = True
                    break  # 更新后退出循环
            elif typecode == key:  # 如果键是单个值
                row[1] = rate
                area_rate_updated = True
                break  # 更新后退出循环
        
        if not area_rate_updated:
            # 如果typecode没有匹配任何规则，可以选择设置一个默认值
            row[1] = 0  # 或者保留原始值，根据需要决定
        
        cursor.updateRow(row)

print("Area_Rate字段更新完成")


Area_Rate字段更新完成


In [22]:
# 更新公众认知度评分字段
with arcpy.da.UpdateCursor(shapefile_A, ["typecode", "Popularity_Rate"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('14'):
            row[1] = 0.6706
        elif typecode.startswith('08'):
            row[1] = 0.501
        elif typecode.startswith('09'):
            row[1] = 0.5069
        elif typecode.startswith('13'):
            row[1] = 0.355
        cursor.updateRow(row)

print("Popularity_Rate更新完成 for 类型 A")

Popularity_Rate更新完成 for 类型 A


In [23]:
# 计算权重字段
expression = "(!Area_Rate! + !Popularity_Rate!) / 2"  
arcpy.CalculateField_management(shapefile_A, 'Weight', expression, "PYTHON3")
print("权重字段更新完成 for 类型 A")

权重字段更新完成 for 类型 A


##### 商业用地B

In [1]:
shapefile_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_B"

In [2]:
# 添加字段
fields_to_add = [
    ('Area_Rate', 'FLOAT'),
    ('Popularity_Rate', 'FLOAT'),
    ('Weight', 'FLOAT')
]

for field_name, field_type in fields_to_add:
    arcpy.AddField_management(shapefile_B, field_name, field_type)
    
print("完成字段添加 for 类型 B")

完成字段添加 for 类型 B


In [3]:
# 更新面积评分字段

# 定义映射，将typecode范围与评分值关联
typecode_to_arearate = {
    tuple([f"{num:06d}" for num in range(50200, 50218)]): 0.01,
    tuple([f"{num:06d}" for num in range(50100, 50124)]): 0.01,
    tuple([f"{num:06d}" for num in range(100100, 100106)]): 0.1,
    tuple([f"{num:06d}" for num in range(160100, 160153)]): 0.1,
    tuple([f"{num:06d}" for num in range(160500, 160502)]): 0.1,
    tuple([f"{num:06d}" for num in range(160400, 160409)]): 0.1,
    tuple([f"{num:06d}" for num in range(60400, 60416)]): 0.8,
    tuple([f"{num:06d}" for num in range(60100, 60104)]): 0.8,
    tuple([f"{num:06d}" for num in range(61000, 61002)]): 0.8,
    "120201": 1,
    "120203": 1
}


# 使用 arcpy.da.UpdateCursor 更新Area_Rate字段
fields = ['typecode', 'Area_Rate']  # 需要处理的字段
with arcpy.da.UpdateCursor(shapefile_B, fields) as cursor:
    for row in cursor:
        typecode = row[0]
        area_rate_updated = False
        
        # 遍历映射规则
        for key, rate in typecode_to_arearate.items():
            if isinstance(key, tuple):  # 如果键是元组（即范围）
                if typecode in key:
                    row[1] = rate
                    area_rate_updated = True
                    break  # 更新后退出循环
            elif typecode == key:  # 如果键是单个值
                row[1] = rate
                area_rate_updated = True
                break  # 更新后退出循环
        
        if not area_rate_updated:
            # 如果typecode没有匹配任何规则，可以选择设置一个默认值
            row[1] = 0  # 或者保留原始值，根据需要决定
        
        cursor.updateRow(row)

print("Area_Rate字段更新完成")


Area_Rate字段更新完成


In [4]:
with arcpy.da.UpdateCursor(shapefile_B, ["typecode", "Popularity_Rate"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('05') or typecode.startswith('10'):
            row[1] = 0.5562
        elif typecode.startswith('06'):
            row[1] = 0.8146
        elif typecode.startswith('16') or typecode.startswith('12'):
            row[1] = 0.3057
        cursor.updateRow(row)

print("Popularity_Rate更新完成 for 类型 B")

Popularity_Rate更新完成 for 类型 B


In [5]:
# 计算权重字段
expression = "(!Area_Rate! + !Popularity_Rate!) / 2"  
arcpy.CalculateField_management(shapefile_B, 'Weight', expression, "PYTHON3")
print("权重字段更新完成 for 类型 B")

权重字段更新完成 for 类型 B


#####  工业用地M

In [6]:
shapefile_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_M"

In [9]:
# 添加字段
fields_to_add = [
    ('Area_Rate', 'FLOAT'),
    ('Popularity_Rate', 'FLOAT'),
    ('Weight', 'FLOAT')
]

for field_name, field_type in fields_to_add:
    arcpy.AddField_management(shapefile_M, field_name, field_type)
    
print("完成字段添加 for 类型 M")

完成字段添加 for 类型 M


In [10]:
# 更新面积评分字段

# 定义映射，将typecode范围与评分值关联
typecode_to_arearate = {
    tuple([f"{num:06d}" for num in range(170200, 170210)]): 0.8,
    "170300": 0.8,
    "120100": 1
}


# 使用 arcpy.da.UpdateCursor 更新Area_Rate字段
fields = ['typecode', 'Area_Rate']  # 需要处理的字段
with arcpy.da.UpdateCursor(shapefile_M, fields) as cursor:
    for row in cursor:
        typecode = row[0]
        area_rate_updated = False
        
        # 遍历映射规则
        for key, rate in typecode_to_arearate.items():
            if isinstance(key, tuple):  # 如果键是元组（即范围）
                if typecode in key:
                    row[1] = rate
                    area_rate_updated = True
                    break  # 更新后退出循环
            elif typecode == key:  # 如果键是单个值
                row[1] = rate
                area_rate_updated = True
                break  # 更新后退出循环
        
        if not area_rate_updated:
            # 如果typecode没有匹配任何规则，可以选择设置一个默认值
            row[1] = 0  # 或者保留原始值，根据需要决定
        
        cursor.updateRow(row)

print("Area_Rate字段更新完成")


Area_Rate字段更新完成


In [11]:
with arcpy.da.UpdateCursor(shapefile_M, ["Popularity_Rate"]) as cursor:
    for row in cursor:
        row[0] = 0.3057  # M类型的权重为0.3057
        cursor.updateRow(row)

print("Popularity_Rate更新完成 for 类型 M")

Popularity_Rate更新完成 for 类型 M


In [12]:
# 计算权重字段
expression = "(!Area_Rate! + !Popularity_Rate!) / 2"  
arcpy.CalculateField_management(shapefile_M, 'Weight', expression, "PYTHON3")
print("权重字段更新完成 for 类型 M")

权重字段更新完成 for 类型 M


##### 交通用地S

In [13]:
shapefile_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_S"

In [14]:
# 添加字段
fields_to_add = [
    ('Area_Rate', 'FLOAT'),
    ('Popularity_Rate', 'FLOAT'),
    ('Weight', 'FLOAT')
]

for field_name, field_type in fields_to_add:
    arcpy.AddField_management(shapefile_S, field_name, field_type)
    
print("完成字段添加 for 类型 S")

完成字段添加 for 类型 S


In [15]:
# 更新面积评分字段

# 定义映射，将typecode范围与评分值关联
typecode_to_arearate = {
    "150200": 1,
    "150104": 1,
    "150300": 1,
    "180200": 0.1,
    "180300": 0.3
}


# 使用 arcpy.da.UpdateCursor 更新Area_Rate字段
fields = ['typecode', 'Area_Rate']  # 需要处理的字段
with arcpy.da.UpdateCursor(shapefile_S, fields) as cursor:
    for row in cursor:
        typecode = row[0]
        area_rate_updated = False
        
        # 遍历映射规则
        for key, rate in typecode_to_arearate.items():
            if isinstance(key, tuple):  # 如果键是元组（即范围）
                if typecode in key:
                    row[1] = rate
                    area_rate_updated = True
                    break  # 更新后退出循环
            elif typecode == key:  # 如果键是单个值
                row[1] = rate
                area_rate_updated = True
                break  # 更新后退出循环
        
        if not area_rate_updated:
            # 如果typecode没有匹配任何规则，可以选择设置一个默认值
            row[1] = 0  # 或者保留原始值，根据需要决定
        
        cursor.updateRow(row)

print("Area_Rate字段更新完成")

Area_Rate字段更新完成


In [16]:
with arcpy.da.UpdateCursor(shapefile_S, ["typecode", "Popularity_Rate"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('15'):
            row[1] = 1.0
        elif typecode.startswith('18'):
            row[1] = 0.01
        cursor.updateRow(row)

print("Popularity_Rate更新完成 for 类型 S")

Popularity_Rate更新完成 for 类型 S


In [17]:
# 计算权重字段
expression = "(!Area_Rate! + !Popularity_Rate!) / 2"  
arcpy.CalculateField_management(shapefile_S, 'Weight', expression, "PYTHON3")
print("权重字段更新完成 for 类型 S")

权重字段更新完成 for 类型 S


##### 公园绿地G

In [19]:
shapefile_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_G"

In [20]:
# 添加字段
fields_to_add = [
    ('Area_Rate', 'FLOAT'),
    ('Popularity_Rate', 'FLOAT'),
    ('Weight', 'FLOAT')
]

for field_name, field_type in fields_to_add:
    arcpy.AddField_management(shapefile_G, field_name, field_type)
    
print("完成字段添加 for 类型 G")

完成字段添加 for 类型 G


In [21]:
# 更新面积评分字段

# 定义映射，将typecode范围与评分值关联
typecode_to_arearate = {
    tuple([f"{num:06d}" for num in range(110100, 110106)]): 0.8,
    tuple([f"{num:06d}" for num in range(110200, 110211)]): 1
}


# 使用 arcpy.da.UpdateCursor 更新Area_Rate字段
fields = ['typecode', 'Area_Rate']  # 需要处理的字段
with arcpy.da.UpdateCursor(shapefile_G, fields) as cursor:
    for row in cursor:
        typecode = row[0]
        area_rate_updated = False
        
        # 遍历映射规则
        for key, rate in typecode_to_arearate.items():
            if isinstance(key, tuple):  # 如果键是元组（即范围）
                if typecode in key:
                    row[1] = rate
                    area_rate_updated = True
                    break  # 更新后退出循环
            elif typecode == key:  # 如果键是单个值
                row[1] = rate
                area_rate_updated = True
                break  # 更新后退出循环
        
        if not area_rate_updated:
            # 如果typecode没有匹配任何规则，可以选择设置一个默认值
            row[1] = 0  # 或者保留原始值，根据需要决定
        
        cursor.updateRow(row)

print("Area_Rate字段更新完成")

Area_Rate字段更新完成


In [22]:
with arcpy.da.UpdateCursor(shapefile_G, ["typecode", "Popularity_Rate"]) as cursor:
    for row in cursor:
        typecode = row[0]
        if typecode.startswith('1101'):
            row[1] = 0.6548
        elif typecode.startswith('1102'):
            row[1] = 0.8245
        cursor.updateRow(row)

print("Popularity_Rate更新完成 for 类型 G")

Popularity_Rate更新完成 for 类型 G


In [23]:
# 计算权重字段
expression = "(!Area_Rate! + !Popularity_Rate!) / 2"  
arcpy.CalculateField_management(shapefile_G, 'Weight', expression, "PYTHON3")
print("权重字段更新完成 for 类型 G")

权重字段更新完成 for 类型 G


#### 步骤四：构建POI密度字段

#### 空间联结

In [26]:
block_layer = r"E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial"

##### 居住用地R

In [27]:
# 对类型 R 的 POI 进行空间连接
poi_layer_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_R"  # 类型 R 的 POI 图层
output_layer_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_R"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_R, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_R, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 R")

空间连接完成 for 类型 R


对于错误落在路网空间上的点，要将离其最近的街区ID与之对应

In [32]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_null_blockid_R = "Null_BlockID_Layer_R"
join_layer_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_R"
arcpy.MakeFeatureLayer_management(join_layer_R, poi_layer_null_blockid_R,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_R, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_R, ["NEAR_FID", "Block_ID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "Block_ID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR R")

更新 BlockID 完成 FOR R


In [35]:
# 保存临时图层
# 指定保存新图层的路径和文件名
output_feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_R"

# 使用CopyFeatures_management函数保存图层
arcpy.CopyFeatures_management("Null_BlockID_Layer_R", output_feature_class)

print(f"图层已保存到 {output_feature_class}")


图层已保存到 E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_R


##### 公服用地A

In [36]:
# 对类型 A 的 POI 进行空间连接
poi_layer_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_A"  # 类型 A 的 POI 图层
output_layer_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_A"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_A, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_A, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 A")

空间连接完成 for 类型 A


In [37]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_null_blockid_A = "Null_BlockID_Layer_A"
join_layer_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_A"
arcpy.MakeFeatureLayer_management(join_layer_A, poi_layer_null_blockid_A,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_A, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_A, ["NEAR_FID", "Block_ID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "Block_ID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR A")

更新 BlockID 完成 FOR A


In [38]:
# 保存临时图层
# 指定保存新图层的路径和文件名
output_feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_A"

# 使用CopyFeatures_management函数保存图层
arcpy.CopyFeatures_management("Null_BlockID_Layer_A", output_feature_class)

print(f"图层已保存到 {output_feature_class}")


图层已保存到 E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_A


##### 商业用地B

In [39]:
# 对类型 B 的 POI 进行空间连接
poi_layer_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_B"  # 类型 B 的 POI 图层
output_layer_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_B"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_B, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_B, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 B")

空间连接完成 for 类型 B


In [40]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_null_blockid_B = "Null_BlockID_Layer_B"
join_layer_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_B"
arcpy.MakeFeatureLayer_management(join_layer_B, poi_layer_null_blockid_B,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_B, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_B, ["NEAR_FID", "Block_ID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "Block_ID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR B")

更新 BlockID 完成 FOR B


In [41]:
# 保存临时图层
# 指定保存新图层的路径和文件名
output_feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_B"

# 使用CopyFeatures_management函数保存图层
arcpy.CopyFeatures_management("Null_BlockID_Layer_B", output_feature_class)

print(f"图层已保存到 {output_feature_class}")


图层已保存到 E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_B


##### 工业用地M

In [42]:
# 对类型 M 的 POI 进行空间连接
poi_layer_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_M"  # 类型 M 的 POI 图层
output_layer_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_M"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_M, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_M, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 M")

空间连接完成 for 类型 M


In [43]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_null_blockid_M = "Null_BlockID_Layer_M"
join_layer_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_M"
arcpy.MakeFeatureLayer_management(join_layer_M, poi_layer_null_blockid_M,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_M, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_M, ["NEAR_FID", "Block_ID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "Block_ID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR M")

更新 BlockID 完成 FOR M


In [44]:
# 保存临时图层
# 指定保存新图层的路径和文件名
output_feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_M"

# 使用CopyFeatures_management函数保存图层
arcpy.CopyFeatures_management("Null_BlockID_Layer_M", output_feature_class)

print(f"图层已保存到 {output_feature_class}")


图层已保存到 E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_M


##### 公园绿地G

In [45]:
# 对类型 G 的 POI 进行空间连接
poi_layer_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_G"  # 类型 G 的 POI 图层
output_layer_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_G"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_G, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_G, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 G")

空间连接完成 for 类型 G


In [49]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_null_blockid_G = "Null_BlockID_Layer_G"
join_layer_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_G"
arcpy.MakeFeatureLayer_management(join_layer_G, poi_layer_null_blockid_G,'"Join_Count" = 0')

# 指定保存新图层的路径和文件名
output_feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_G"

# 使用CopyFeatures_management函数保存图层
arcpy.CopyFeatures_management("Null_BlockID_Layer_G", output_feature_class)

print(f"图层已保存到 {output_feature_class}")

# 删除这个临时图层中的所有要素
arcpy.DeleteFeatures_management(poi_layer_null_blockid_G)

print("已删除 Join_Count 为 0 的记录")


图层已保存到 E:\stu1\Projects\POI2023\POI2023.gdb\poi23\Null_G
已删除 Join_Count 为 0 的记录


##### 交通用地S

In [47]:
# 对类型 S 的 POI 进行空间连接
poi_layer_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\POI_S"  # 类型 S 的 POI 图层
output_layer_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_S"  # 输出图层

# 执行空间连接
arcpy.SpatialJoin_analysis(target_features=poi_layer_S, 
                           join_features=block_layer, 
                           out_feature_class=output_layer_S, 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL")

print("空间连接完成 for 类型 S")

空间连接完成 for 类型 S


In [48]:
# 创建一个 BlockID 为 Null 的 POI 点的图层
poi_layer_null_blockid_S = "Null_BlockID_Layer_S"
join_layer_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_S"
arcpy.MakeFeatureLayer_management(join_layer_S, poi_layer_null_blockid_S,'"Join_Count" = 0')

# 使用 Near Analysis 找到最近的街区
arcpy.analysis.Near(poi_layer_null_blockid_S, block_layer)

# 更新 BlockID 为最近街区的 BlockID
with arcpy.da.UpdateCursor(poi_layer_null_blockid_S, ["NEAR_FID", "Block_ID"]) as cursor:
    for row in cursor:
        # 获取最近街区的 FID
        near_fid = row[0]

        # 查找对应街区的 BlockID
        with arcpy.da.SearchCursor(block_layer, ["OBJECTID", "Block_ID"]) as block_cursor:
            for block_row in block_cursor:
                if block_row[0] == near_fid:
                    row[1] = block_row[1]
                    cursor.updateRow(row)
                    break

print("更新 BlockID 完成 FOR S")

更新 BlockID 完成 FOR S


#### 频率密度字段(未加权）

In [50]:
# 准备工作：定义街区图层和POI图层
block_layer = r"E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial"
poi_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_R"

# 第1步：计算每个街区内的POI点个数
block_poi_count = {}
with arcpy.da.SearchCursor(poi_R, ["Block_ID"]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_poi_count:
            block_poi_count[block_id] += 1
        else:
            block_poi_count[block_id] = 1

# 第2步：计算总的POI点个数
total_poi_count = sum(block_poi_count.values())

# 第3步：创建新表格来存储街区的频率密度
output_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\R_Frequency_Density"
arcpy.CreateTable_management(r"E:\stu1\Projects\POI2023\POI2023.gdb", "R_Frequency_Density")
arcpy.AddField_management(output_table, "Block_ID", "LONG")
arcpy.AddField_management(output_table, "Frequency_Density", "FLOAT")

# 第4步：填充新表格
with arcpy.da.InsertCursor(output_table, ["Block_ID", "Frequency_Density"]) as insert_cursor:
    for block_id, poi_count in block_poi_count.items():
        frequency_density = poi_count / total_poi_count
        insert_cursor.insertRow([block_id, frequency_density])

print("频率密度表R创建并填充完成")


频率密度表R创建并填充完成


In [51]:
poi_A = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_A"

# 第1步：计算每个街区内的POI点个数
block_poi_count = {}
with arcpy.da.SearchCursor(poi_A, ["Block_ID"]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_poi_count:
            block_poi_count[block_id] += 1
        else:
            block_poi_count[block_id] = 1

# 第2步：计算总的POI点个数
total_poi_count = sum(block_poi_count.values())

# 第3步：创建新表格来存储街区的频率密度
output_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\A_Frequency_Density"
arcpy.CreateTable_management(r"E:\stu1\Projects\POI2023\POI2023.gdb", "A_Frequency_Density")
arcpy.AddField_management(output_table, "Block_ID", "LONG")
arcpy.AddField_management(output_table, "Frequency_Density", "FLOAT")

# 第4步：填充新表格
with arcpy.da.InsertCursor(output_table, ["Block_ID", "Frequency_Density"]) as insert_cursor:
    for block_id, poi_count in block_poi_count.items():
        frequency_density = poi_count / total_poi_count
        insert_cursor.insertRow([block_id, frequency_density])

print("频率密度表A创建并填充完成")


频率密度表A创建并填充完成


In [52]:
poi_B = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_B"

# 第1步：计算每个街区内的POI点个数
block_poi_count = {}
with arcpy.da.SearchCursor(poi_B, ["Block_ID"]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_poi_count:
            block_poi_count[block_id] += 1
        else:
            block_poi_count[block_id] = 1

# 第2步：计算总的POI点个数
total_poi_count = sum(block_poi_count.values())

# 第3步：创建新表格来存储街区的频率密度
output_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\B_Frequency_Density"
arcpy.CreateTable_management(r"E:\stu1\Projects\POI2023\POI2023.gdb", "B_Frequency_Density")
arcpy.AddField_management(output_table, "Block_ID", "LONG")
arcpy.AddField_management(output_table, "Frequency_Density", "FLOAT")

# 第4步：填充新表格
with arcpy.da.InsertCursor(output_table, ["Block_ID", "Frequency_Density"]) as insert_cursor:
    for block_id, poi_count in block_poi_count.items():
        frequency_density = poi_count / total_poi_count
        insert_cursor.insertRow([block_id, frequency_density])

print("频率密度表B创建并填充完成")


频率密度表B创建并填充完成


In [53]:
poi_M = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_M"

# 第1步：计算每个街区内的POI点个数
block_poi_count = {}
with arcpy.da.SearchCursor(poi_M, ["Block_ID"]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_poi_count:
            block_poi_count[block_id] += 1
        else:
            block_poi_count[block_id] = 1

# 第2步：计算总的POI点个数
total_poi_count = sum(block_poi_count.values())

# 第3步：创建新表格来存储街区的频率密度
output_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\M_Frequency_Density"
arcpy.CreateTable_management(r"E:\stu1\Projects\POI2023\POI2023.gdb", "M_Frequency_Density")
arcpy.AddField_management(output_table, "Block_ID", "LONG")
arcpy.AddField_management(output_table, "Frequency_Density", "FLOAT")

# 第4步：填充新表格
with arcpy.da.InsertCursor(output_table, ["Block_ID", "Frequency_Density"]) as insert_cursor:
    for block_id, poi_count in block_poi_count.items():
        frequency_density = poi_count / total_poi_count
        insert_cursor.insertRow([block_id, frequency_density])

print("频率密度表M创建并填充完成")


频率密度表M创建并填充完成


In [54]:
poi_G = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_G"

# 第1步：计算每个街区内的POI点个数
block_poi_count = {}
with arcpy.da.SearchCursor(poi_G, ["Block_ID"]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_poi_count:
            block_poi_count[block_id] += 1
        else:
            block_poi_count[block_id] = 1

# 第2步：计算总的POI点个数
total_poi_count = sum(block_poi_count.values())

# 第3步：创建新表格来存储街区的频率密度
output_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\G_Frequency_Density"
arcpy.CreateTable_management(r"E:\stu1\Projects\POI2023\POI2023.gdb", "G_Frequency_Density")
arcpy.AddField_management(output_table, "Block_ID", "LONG")
arcpy.AddField_management(output_table, "Frequency_Density", "FLOAT")

# 第4步：填充新表格
with arcpy.da.InsertCursor(output_table, ["Block_ID", "Frequency_Density"]) as insert_cursor:
    for block_id, poi_count in block_poi_count.items():
        frequency_density = poi_count / total_poi_count
        insert_cursor.insertRow([block_id, frequency_density])

print("频率密度表G创建并填充完成")


频率密度表G创建并填充完成


In [55]:
poi_S = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_S"

# 第1步：计算每个街区内的POI点个数
block_poi_count = {}
with arcpy.da.SearchCursor(poi_S, ["Block_ID"]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_poi_count:
            block_poi_count[block_id] += 1
        else:
            block_poi_count[block_id] = 1

# 第2步：计算总的POI点个数
total_poi_count = sum(block_poi_count.values())

# 第3步：创建新表格来存储街区的频率密度
output_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\S_Frequency_Density"
arcpy.CreateTable_management(r"E:\stu1\Projects\POI2023\POI2023.gdb", "S_Frequency_Density")
arcpy.AddField_management(output_table, "Block_ID", "LONG")
arcpy.AddField_management(output_table, "Frequency_Density", "FLOAT")

# 第4步：填充新表格
with arcpy.da.InsertCursor(output_table, ["Block_ID", "Frequency_Density"]) as insert_cursor:
    for block_id, poi_count in block_poi_count.items():
        frequency_density = poi_count / total_poi_count
        insert_cursor.insertRow([block_id, frequency_density])

print("频率密度表S创建并填充完成")


频率密度表S创建并填充完成


#### 频率密度字段（用权重调整点数后）

In [1]:
import arcpy

# 设置环境
arcpy.env.workspace = r'E:\stu1\Projects\POI2023\POI2023.gdb'
arcpy.env.overwriteOutput = True

**首先对于类型A**

In [2]:
# 定义输入和输出
input_poi_layer = 'JOIN_A'
weight_field = 'Weight'  # POI的权重字段
block_id_field = 'Block_ID'  # 街区标识字段

In [7]:
# 街区要素类
block_layer = 'blocks_initial'
frequency_density_field = 'FD_A' # 将要创建的频率密度字段

In [11]:
# 初始化用于汇总每个街区加权POI点数的字典
block_weights = {}
total_weight = 0

In [12]:
# 第1步：按街区代码汇总POI的权重
with arcpy.da.SearchCursor(input_poi_layer, [block_id_field, weight_field]) as cursor:
    for row in cursor:
        block_id, weight = row
        block_weights[block_id] = block_weights.get(block_id, 0) + weight
        total_weight += weight

In [13]:
# 调试输出：汇总后的权重和总权重
print("Block weights:", block_weights)
print("Total weight:", total_weight)

Block weights: {726: 1569.6450080871582, 841: 4096.515033721924, 482: 2696.670021057129, 644: 2700.7650623321533, 575: 1155.7800025939941, 565: 6142.800136566162, 257: 2767.6750984191895, 250: 8624.289974212646, 525: 4036.810031890869, 359: 6264.965118408203, 766: 3107.430013656616, 800: 1390.02001953125, 944: 2294.2650146484375, 298: 4775.1700439453125, 348: 3448.2350940704346, 330: 11351.0202293396, 491: 3587.7950744628906, 681: 4916.285022735596, 271: 10289.085037231445, 214: 4023.149990081787, 723: 3889.345069885254, 642: 461.9850044250488, 690: 2252.260019302368, 329: 2137.3900260925293, 441: 1702.4350128173828, 240: 29781.14017677307, 918: 2590.7651138305664, 821: 439.8250160217285, 836: 3126.955135345459, 159: 194.1700096130371, 220: 2054.3450241088867, 666: 563.2249984741211, 511: 2051.7500171661377, 173: 2875.505100250244, 657: 6340.620067596436, 399: 4508.510025024414, 896: 6456.265022277832, 364: 2244.195095062256, 419: 40.04999923706055, 560: 370.700008392334, 49: 1790.3450


Total weight: 1431806.0685749054


In [14]:
# 第2步：计算每个街区的频率密度
block_freq_density = {block: weight / total_weight for block, weight in block_weights.items()}

In [15]:
# 调试输出：计算出的频率密度值
for block_id, freq_density in block_freq_density.items():
    print(f"Block ID: {block_id}, Frequency Density: {freq_density}")

Block ID: 726, Frequency Density: 0.0010962692801333392
Block ID: 841, Frequency Density: 0.00286108232366918
Block ID: 482, Frequency Density: 0.001883404519818217
Block ID: 644, Frequency Density: 0.0018862645728413897
Block ID: 575, Frequency Density: 0.0008072182594842306
Block ID: 565, Frequency Density: 0.004290245914853656
Block ID: 257, Frequency Density: 0.0019329957870439056
Block ID: 250, Frequency Density: 0.006023364590706415
Block ID: 525, Frequency Density: 0.002819383239455576
Block ID: 359, Frequency Density: 0.004375568211303785
Block ID: 766, Frequency Density: 0.002170286941687208
Block ID: 800, Frequency Density: 0.0009708158458322184
Block ID: 944, Frequency Density: 0.0016023573757666415
Block ID: 298, Frequency Density: 0.003335067610586467
Block ID: 348, Frequency Density: 0.0024083115512301932
Block ID: 330, Frequency Density: 0.0079277637373317
Block ID: 491, Frequency Density: 0.0025057828383377844
Block ID: 681, Frequency Density: 0.0034336249375090553
Bloc

Block ID: 809, Frequency Density: 0.0006430794216283661
Block ID: 365, Frequency Density: 0.0005966555332999985
Block ID: 920, Frequency Density: 0.0015087623020705106
Block ID: 207, Frequency Density: 0.001431538140915041
Block ID: 543, Frequency Density: 0.00021601040144350318
Block ID: 588, Frequency Density: 0.0013190089543406549
Block ID: 879, Frequency Density: 0.0013056691546829218
Block ID: 808, Frequency Density: 0.000853596751562271
Block ID: 688, Frequency Density: 0.0004361833824741981
Block ID: 979, Frequency Density: 0.002210023499294769
Block ID: 519, Frequency Density: 0.001087092060062798
Block ID: 834, Frequency Density: 0.00044112119648448975
Block ID: 904, Frequency Density: 0.002282019974103911
Block ID: 272, Frequency Density: 0.0010108736580419465
Block ID: 750, Frequency Density: 0.0011411531674755655
Block ID: 332, Frequency Density: 0.0010578038908019667
Block ID: 754, Frequency Density: 0.0013819329716038894
Block ID: 496, Frequency Density: 0.001460742527331

Block ID: 672, Frequency Density: 0.00040263833762947156
Block ID: 533, Frequency Density: 0.0007040234307727619
Block ID: 457, Frequency Density: 0.0010970096101444705
Block ID: 884, Frequency Density: 0.0006758282590302982
Block ID: 991, Frequency Density: 0.0004031830926333976
Block ID: 52, Frequency Density: 0.00018796540312940633
Block ID: 393, Frequency Density: 0.0010946349884790058
Block ID: 674, Frequency Density: 0.0004875765203640992
Block ID: 1035, Frequency Density: 0.0002575418666833952
Block ID: 724, Frequency Density: 0.00041486415609212006
Block ID: 65, Frequency Density: 0.0006735758734094966
Block ID: 1016, Frequency Density: 0.0003069130770576239
Block ID: 114, Frequency Density: 0.0006635116560597621
Block ID: 144, Frequency Density: 0.0003400460601461236
Block ID: 325, Frequency Density: 0.00015568100408357087
Block ID: 633, Frequency Density: 0.0006491347016800024
Block ID: 771, Frequency Density: 0.0003962861955673619
Block ID: 431, Frequency Density: 0.00015485

Block ID: 280, Frequency Density: 0.00014217008728458757
Block ID: 232, Frequency Density: 0.00010113799944808324
Block ID: 299, Frequency Density: 0.00020425951850382092
Block ID: 360, Frequency Density: 7.931241797733035e-05
Block ID: 183, Frequency Density: 0.00013900625451140458
Block ID: 997, Frequency Density: 0.0003364142775280281
Block ID: 30, Frequency Density: 0.00016133469823180537
Block ID: 315, Frequency Density: 0.00023316007055310242
Block ID: 381, Frequency Density: 4.033367455795134e-05
Block ID: 283, Frequency Density: 0.00013749767575522745
Block ID: 317, Frequency Density: 4.033367455795134e-05
Block ID: 25, Frequency Density: 0.0002466465374788976
Block ID: 213, Frequency Density: 0.0002735566082207709
Block ID: 719, Frequency Density: 6.72437452998246e-05
Block ID: 469, Frequency Density: 6.72437452998246e-05
Block ID: 246, Frequency Density: 9.867257995345322e-05
Block ID: 19, Frequency Density: 0.0002108316268327016
Block ID: 279, Frequency Density: 0.0001210638

In [16]:
# 第3步：在街区要素类中创建频率密度字段（如果尚不存在）
if len(arcpy.ListFields(block_layer, frequency_density_field)) == 0:
    arcpy.AddField_management(block_layer, frequency_density_field, "DOUBLE")

In [17]:
# 更新街区要素类中的频率密度字段
with arcpy.da.UpdateCursor(block_layer, [block_id_field, frequency_density_field]) as cursor:
    for row in cursor:
        block_id = row[0]
        if block_id in block_freq_density:
            row[1] = block_freq_density[block_id]  # 更新频率密度
        else:
            row[1] = 0  # 对于没有POI点的街区，频率密度设置为0
        cursor.updateRow(row)

print("街区要素类中的频率密度字段更新完成。")

街区要素类中的频率密度字段更新完成。


**尝试定义函数**

In [18]:
def calculate_and_update_frequency_density(input_poi_layer, weight_field, block_id_field, block_layer, frequency_density_field):
    # 初始化用于汇总每个街区加权POI点数的字典
    block_weights = {}
    total_weight = 0

    # 第1步：按街区代码汇总POI的权重
    with arcpy.da.SearchCursor(input_poi_layer, [block_id_field, weight_field]) as cursor:
        for row in cursor:
            block_id, weight = row
            block_weights[block_id] = block_weights.get(block_id, 0) + weight
            total_weight += weight

    # 第2步：计算每个街区的频率密度
    block_freq_density = {block: weight / total_weight for block, weight in block_weights.items()}

    # 第3步：在街区要素类中创建频率密度字段（如果尚不存在）
    if len(arcpy.ListFields(block_layer, frequency_density_field)) == 0:
        arcpy.AddField_management(block_layer, frequency_density_field, "DOUBLE")

    # 更新街区要素类中的频率密度字段
    with arcpy.da.UpdateCursor(block_layer, [block_id_field, frequency_density_field]) as cursor:
        for row in cursor:
            block_id = row[0]
            if block_id in block_freq_density:
                row[1] = block_freq_density[block_id]  # 更新频率密度
            else:
                row[1] = 0  # 对于没有POI点的街区，频率密度设置为0
            cursor.updateRow(row)

    print("街区要素类中的频率密度字段更新完成。")

In [19]:
# 使用定义的函数处理数据
calculate_and_update_frequency_density('JOIN_R', 'Weight', 'Block_ID', 'blocks_initial', 'FD_R')

街区要素类中的频率密度字段更新完成。


**此处，权重的调整影响因子并重要**

In [20]:
calculate_and_update_frequency_density('JOIN_B', 'Weight', 'Block_ID', 'blocks_initial', 'FD_B')

街区要素类中的频率密度字段更新完成。


In [21]:
calculate_and_update_frequency_density('JOIN_M', 'Weight', 'Block_ID', 'blocks_initial', 'FD_M')

街区要素类中的频率密度字段更新完成。


In [22]:
calculate_and_update_frequency_density('JOIN_G', 'Weight', 'Block_ID', 'blocks_initial', 'FD_G')

街区要素类中的频率密度字段更新完成。


In [23]:
calculate_and_update_frequency_density('JOIN_S', 'Weight', 'Block_ID', 'blocks_initial', 'FD_S')

街区要素类中的频率密度字段更新完成。


#### 核密度字段

**首先要投影POI数据**

In [30]:
input_features = "JOIN_S"

In [31]:
factor = 100
arcpy.CalculateField_management(input_features, "Weight", f"!Weight! * {factor}", "PYTHON3")

<Result 'JOIN_S'>

###### 步骤1：核密度分析

In [1]:
import arcpy
from arcpy.sa import *

# 检查空间分析扩展是否可用
arcpy.CheckOutExtension("Spatial")

# 设置环境变量
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"  # 替换为您的工作空间路径
arcpy.env.overwriteOutput = True

调整权重字段

In [6]:
input_features = "JOIN_A" 

为POI数据投影

In [12]:
# 输入POI图层
input_features = "JOIN_A_Project"
population_field = "Weight"  # 若存在权重字段则替换，否则设置为None
# 设置搜索半径（带宽）和输出栅格的单元格尺寸（输出分辨率）
search_radius = "500 Meters"  # 减小带宽，例如设置为500米
cell_size = "10"  # 减小栅格单元格尺寸，例如设置为10米

output_raster = "KernelDensity_Output_A"  # 输出核密度栅格图层的名称

# 执行核密度分析
kd_result = KernelDensity(input_features, population_field, cell_size, search_radius, area_unit_scale_factor="SQUARE_METERS")
kd_result.save(output_raster)

print("核密度分析完成，输出保存至", output_raster)

核密度分析完成，输出保存至 KernelDensity_Output_A


<geoprocessing server result object object at 0x0000025368368090>

##### 核密度估计

In [1]:
import arcpy
from arcpy.sa import *

# 确保空间分析扩展可用
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [2]:
# 设置环境
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"  # 指定工作空间路径
arcpy.env.overwriteOutput = True

###### 以居住用地R为例

In [36]:
# 核密度分析
input_features = "JOIN_R_Project"  # 指定输入的POI图层
population_field = "Weight"  # 使用Weight字段作为权重
search_radius = 1000  # 居住用地的服务半径为1000米
output_raster = "KernelDensity_R"  # 输出栅格图层的名称

In [37]:
# 执行核密度分析
kd_result = KernelDensity(input_features, population_field="Weight",cell_size=10, search_radius=search_radius, area_unit_scale_factor="SQUARE_METERS")
kd_result.save(output_raster)

print("R类核密度分析完成")

R类核密度分析完成


In [3]:
# 街区要素类
block_feature_class = "blocks_initial"

In [4]:
# 新字段名称
new_field_name = "KD_R"

In [5]:
# 添加新字段
arcpy.AddField_management(block_feature_class, new_field_name, "DOUBLE")

<Result 'blocks_initial'>

In [7]:
# 核密度分析结果栅格
kd_raster = "KernelDensity_R"

# 街区要素类
block_feature_class = "blocks_initial"
zone_field = "Block_ID"  # 街区要素类中用来标识每个街区的唯一字段

# 输出统计表格路径
out_table = "ZonalStatistics_R"

In [39]:
# 执行区域统计作为表格
ZonalStatisticsAsTable(block_feature_class, zone_field, kd_raster, out_table, "DATA", "SUM")

<geoprocessing server result object object at 0x0000021E1D511120>

In [10]:
# 使用 JoinField 将统计表格的SUM字段添加到街区要素类的属性表中
# 此时字段名称保持为 SUM
arcpy.JoinField_management(block_feature_class, "Block_ID", out_table, "OBJECTID", ["SUM"])


<Result 'blocks_initial'>

In [9]:
# 获取并打印两个数据集中Block_ID的唯一值
block_ids_feature_class = set([row[0] for row in arcpy.da.SearchCursor(block_feature_class, ["Block_ID"]) if row[0] is not None])
block_ids_out_table = set([row[0] for row in arcpy.da.SearchCursor(out_table, ["Block_ID"]) if row[0] is not None])

# 检查两个集合的差异
unmatched_feature_class = block_ids_feature_class.difference(block_ids_out_table)
unmatched_out_table = block_ids_out_table.difference(block_ids_feature_class)

print("在街区要素类中未匹配的Block_ID：", unmatched_feature_class)
print("在统计表中未匹配的Block_ID：", unmatched_out_table)


在街区要素类中未匹配的Block_ID： set()
在统计表中未匹配的Block_ID： set()


In [11]:
# 复制 SUM 字段的值到新字段
with arcpy.da.UpdateCursor(block_feature_class, ["SUM", new_field_name]) as cursor:
    for row in cursor:
        row[1] = row[0]  # 将 SUM 字段的值复制到新字段
        cursor.updateRow(row)

# 如果您不再需要 SUM 字段，可以将其删除
arcpy.DeleteField_management(block_feature_class, "SUM")


<Result 'blocks_initial'>

##### 定义函数完成

In [12]:
import arcpy
from arcpy.sa import *

def perform_kernel_density_and_update_blocks(input_features, population_field, search_radius, cell_size, output_raster_name, block_feature_class, zone_field, new_field_name):
    """
    执行核密度分析，并将结果按街区汇总，最后更新到街区要素类的新字段中。

    参数:
    - input_features: 输入的POI图层路径。
    - population_field: 使用的权重字段名。
    - search_radius: 核密度分析的搜索半径。
    - cell_size: 核密度分析的栅格单元大小。
    - output_raster_name: 输出核密度栅格图层的名称。
    - block_feature_class: 街区要素类路径。
    - zone_field: 街区要素类中用于标识每个街区的字段名。
    - new_field_name: 更新到街区要素类中的新字段名。
    """
    # 确保空间分析扩展可用
    arcpy.CheckOutExtension("Spatial")
    
    # 核密度分析
    kd_result = KernelDensity(input_features, population_field, cell_size, search_radius, area_unit_scale_factor="SQUARE_METERS")
    kd_result.save(output_raster_name)
    print(f"{output_raster_name} 核密度分析完成")
    
    # 执行区域统计作为表格
    out_table = output_raster_name + "_ZonalStats"
    ZonalStatisticsAsTable(block_feature_class, zone_field, output_raster_name, out_table, "DATA", "SUM")
    
    # 使用 JoinField 将统计表格的 SUM 字段添加到街区要素类的属性表中
    arcpy.JoinField_management(block_feature_class, zone_field, out_table, "OBJECTID", ["SUM"])
    
    # 添加新字段（如果尚未存在）
    if len(arcpy.ListFields(block_feature_class, new_field_name)) == 0:
        arcpy.AddField_management(block_feature_class, new_field_name, "DOUBLE")
    
    # 复制 SUM 字段的值到新字段，并删除 SUM 字段
    with arcpy.da.UpdateCursor(block_feature_class, ["SUM", new_field_name]) as cursor:
        for row in cursor:
            row[1] = row[0]  # 将 SUM 字段的值复制到新字段
            cursor.updateRow(row)
    arcpy.DeleteField_management(block_feature_class, "SUM")
    print(f"{new_field_name} 字段已更新")


##### 调用函数

In [13]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_A_Project",
    population_field="Weight",
    search_radius=2000,
    cell_size=10,
    output_raster_name="KernelDensity_A",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KD_A"
)

KernelDensity_A 核密度分析完成
KD_A 字段已更新


In [14]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_B_Project",
    population_field="Weight",
    search_radius=1500,
    cell_size=10,
    output_raster_name="KernelDensity_B",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KD_B"
)


KernelDensity_B 核密度分析完成
KD_B 字段已更新


In [15]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_G_Project",
    population_field="Weight",
    search_radius=2000,
    cell_size=10,
    output_raster_name="KernelDensity_G",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KD_G"
)


KernelDensity_G 核密度分析完成
KD_G 字段已更新


In [16]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_M_Project",
    population_field="Weight",
    search_radius=1500,
    cell_size=10,
    output_raster_name="KernelDensity_M",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KD_M"
)


KernelDensity_M 核密度分析完成
KD_M 字段已更新


In [17]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_S_Project",
    population_field="Weight",
    search_radius=3000,
    cell_size=10,
    output_raster_name="KernelDensity_S",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KD_S"
)


KernelDensity_S 核密度分析完成
KD_S 字段已更新


###### 使用 ZonalStatistics 统计每个多边形区域的栅格总值

In [7]:
# 街区单元图层（包含多个零散的多边形）
zone_features = "blocks_initial"
zone_field = "Block_ID"  # 街区单元的唯一标识字段

# 输出栅格，保存每个区域的统计结果
out_zonal_stats = "ZonalStatistics_R"

# 执行ZonalStatistics
zs_result = ZonalStatistics(zone_features, zone_field, output_raster, "SUM")
zs_result.save(out_zonal_stats)


###### 统计每个街区内的栅格总值

###### 将结果添加到BLOCKS的字段中

In [14]:
# 检查<NULL>字段

field_to_check = "KD_R"

# 初始化计数器
null_count = 0

# 使用 SearchCursor 遍历字段
with arcpy.da.SearchCursor(block_feature_class, [field_to_check]) as cursor:
    for row in cursor:
        # 如果字段值为 None（即在ArcGIS中显示为 <Null>），计数器加一
        if row[0] is None:
            null_count += 1

# 输出结果
print(f"Number of <Null> values in field '{field_to_check}': {null_count}")


Number of <Null> values in field 'KD_R': 1022


In [ ]:
# 输入POI图层和街区单元图层
poi_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_R"
block_layer = r"E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial"

# 输出核密度图层路径
kernel_R = r"E:\stu1\Projects\POI2023\POI2023.gdb\kd_R1"

In [ ]:
# 执行核密度分析，使用Weight字段作为权重
kernel_density = KernelDensity(poi_R, population_field="Weight")
kernel_density.save(kernel_R)

print("核密度分析完成")

In [ ]:
# 输出统计表格路径
output_stats_table = r"E:\stu1\Projects\POI2023\POI2023.gdb\KD_R"

# 对每个街区单元进行统计，计算栅格总值
ZonalStatisticsAsTable(block_layer, "Block_ID", clipped_density, output_stats_table, "DATA", "SUM")

print("每个街区单元内的栅格总值统计完成")


#### 网络核密度字段

In [1]:
import arcpy
from arcpy.sa import *

def KDB2blocks(raster_name, block_feature_class, zone_field, new_field_name):
    """
    执行核密度分析，并将结果按街区汇总，最后更新到街区要素类的新字段中。

    参数:
    - raster_name: 核密度栅格图层的名称。
    - block_feature_class: 街区要素类路径。
    - zone_field: 街区要素类中用于标识每个街区的字段名。
    - new_field_name: 更新到街区要素类中的新字段名。
    """
    
    # 执行区域统计作为表格
    out_table = raster_name + "_ZonalStats_BARR"
    ZonalStatisticsAsTable(block_feature_class, zone_field, raster_name, out_table, "DATA", "SUM")
    
    # 使用 JoinField 将统计表格的 SUM 字段添加到街区要素类的属性表中
    arcpy.JoinField_management(block_feature_class, zone_field, out_table, "OBJECTID", ["SUM"])
    
    # 添加新字段（如果尚未存在）
    if len(arcpy.ListFields(block_feature_class, new_field_name)) == 0:
        arcpy.AddField_management(block_feature_class, new_field_name, "DOUBLE")
    
    # 复制 SUM 字段的值到新字段，并删除 SUM 字段
    with arcpy.da.UpdateCursor(block_feature_class, ["SUM", new_field_name]) as cursor:
        for row in cursor:
            row[1] = row[0]  # 将 SUM 字段的值复制到新字段
            cursor.updateRow(row)
    arcpy.DeleteField_management(block_feature_class, "SUM")
    print(f"{new_field_name} 字段已更新")

In [3]:
KDB2blocks(
    raster_name="KDB_M1",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KDB_M"
)

KDB_M 字段已更新


In [4]:
KDB2blocks(
    raster_name="KDB_G1",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KDB_G"
)

KDB_G 字段已更新


In [5]:
KDB2blocks(
    raster_name="KDB_S1",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KDB_S"
)

KDB_S 字段已更新


In [6]:
KDB2blocks(
    raster_name="KDB_B1",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KDB_B"
)

KDB_B 字段已更新


In [7]:
KDB2blocks(
    raster_name="KDB_R1",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KDB_R"
)

KDB_R 字段已更新


In [8]:
KDB2blocks(
    raster_name="KDB_A1",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="KDB_A"
)

KDB_A 字段已更新


#### 步骤五：构建POI去从聚调整字段

##### 四叉树自适应划分格网

In [20]:
# 四叉树节点类
class QuadTreeNode:
    node_count = 0  # 用于生成唯一的格网ID
    
    def __init__(self, extent, max_points=1000, max_depth=5, depth=0):
        self.extent = extent  # 节点的空间范围
        self.children = []  # 子节点列表
        self.max_points = max_points  # 每个节点能包含的最大点数
        self.max_depth = max_depth  # 树的最大深度
        self.depth = depth  # 当前节点的深度
        self.points = []  # 此节点包含的POI点列表
        self.grid_id = QuadTreeNode.node_count  # 唯一的格网ID
        QuadTreeNode.node_count += 1
        
    # 分割当前节点为四个子节点
    def split(self):
        xmin, ymin, xmax, ymax = self.extent
        xmid, ymid = (xmin + xmax) / 2, (ymin + ymax) / 2
        
        # 定义四个子区域的空间范围
        extents = [
            (xmin, ymin, xmid, ymid),  # 西南
            (xmid, ymin, xmax, ymid),  # 东南
            (xmin, ymid, xmid, ymax),  # 西北
            (xmid, ymid, xmax, ymax)   # 东北
        ]
    
        # 为每个子区域创建一个新的四叉树节点
        for extent in extents:
            self.children.append(QuadTreeNode(extent, max_points=self.max_points, max_depth=self.max_depth, depth=self.depth + 1))
            
    # 将一个点插入四叉树中，并存储格网ID或节点引用
    def insert(self, point):
        if not self._contains(point):
            return False  # 点不在当前节点的范围内

        if len(self.points) < self.max_points or self.depth == self.max_depth:
            self.points.append(point)  # 如果节点未满，或达到最大深度，直接添加点
            return True

        if not self.children:
            self.split()  # 如果子节点不存在，先分割当前节点

        return any(child.insert(point) for child in self.children)
    
    # 检查点是否在当前节点的范围内
    def _contains(self, point):
        x, y = point
        xmin, ymin, xmax, ymax = self.extent
        return xmin <= x < xmax and ymin <= y < ymax

In [21]:
# 构建四叉树并为每个POI点分配格网ID
def build_quadtree_and_assign_grid_ids(poi_layer, root):
    with arcpy.da.UpdateCursor(poi_layer, ["SHAPE@XY", "GridID"]) as cursor:
        for row in cursor:
            point = row[0]
            if root.insert(point):
                # 如果点被插入到四叉树，更新其格网ID
                row[1] = find_grid_id(root, point)
                cursor.updateRow(row)

In [22]:
# 递归查找包含给定点的节点的格网ID
def find_grid_id(node, point):
    if node._contains(point):
        if not node.children or node.depth == node.max_depth:
            return node.grid_id
        for child in node.children:
            grid_id = find_grid_id(child, point)
            if grid_id is not None:
                return grid_id
    return None

##### 定义基本参数

In [24]:
# 根据研究区域边界矢量数据生成四叉树覆盖的空间范围
ring3 = r"E:\stu1\Projects\POI2023\POI2023.gdb\ring3"
boundary_extent = arcpy.Describe(ring3).extent
tree_extent = (boundary_extent.XMin, boundary_extent.YMin, boundary_extent.XMax, boundary_extent.YMax)

In [25]:
# 创建四叉树的根节点
max_points_per_node = 100
max_depth = 7
root = QuadTreeNode(tree_extent, max_points=max_points_per_node, max_depth=max_depth)

##### 去从聚调整

In [26]:
# 计算每个格网内POI的数量
def calculate_grid_poi_counts(poi_layer):
    grid_poi_counts = {}
    with arcpy.da.SearchCursor(poi_layer, ["GridID"]) as cursor:
        for row in cursor:
            grid_id = row[0]
            grid_poi_counts[grid_id] = grid_poi_counts.get(grid_id, 0) + 1
    return grid_poi_counts

In [31]:
# 为每个POI点分配权重
def assign_weights_to_pois(poi_layer, grid_poi_counts):
    # 计算有POI的格网总数
    total_occupied_grids = len(grid_poi_counts)
    # 计算每个格网的权重
    grid_weight = 1.0 / total_occupied_grids if total_occupied_grids else 0
    
    with arcpy.da.UpdateCursor(poi_layer, ["GridID", "Decluster"]) as cursor:
        for row in cursor:
            grid_id = row[0]
            poi_count = grid_poi_counts.get(grid_id, 0)
            Decluster = grid_weight / poi_count if poi_count > 0 else 0
            row[1] = Decluster
            cursor.updateRow(row)

##### 遍历不同类型POI

In [32]:
def process_poi_layer(poi_layer_name, poi_layer):
    print(f"开始处理图层: {poi_layer_name}")
    arcpy.AddField_management(poi_layer, "GridID", "LONG")
    arcpy.AddField_management(poi_layer, "Decluster", "FLOAT")
    build_quadtree_and_assign_grid_ids(poi_layer, root)
    grid_poi_counts = calculate_grid_poi_counts(poi_layer)
    assign_weights_to_pois(poi_layer, grid_poi_counts)
    print(f"图层处理完成: {poi_layer_name}")

In [33]:
# 列出所有待处理的 POI 图层名称及其对应的图层对象
poi_layers = {
    "shapefile_R": shapefile_R,
    "shapefile_A": shapefile_A,
    "shapefile_B": shapefile_B,
    "shapefile_M": shapefile_M,
    "shapefile_S": shapefile_S,
    "shapefile_G": shapefile_G
}

# 对每个图层执行处理，并加上进度说明
for poi_layer_name, poi_layer in poi_layers.items():
    process_poi_layer(poi_layer_name, poi_layer)

开始处理图层: shapefile_R
图层处理完成: shapefile_R
开始处理图层: shapefile_A
图层处理完成: shapefile_A
开始处理图层: shapefile_B
图层处理完成: shapefile_B
开始处理图层: shapefile_M
图层处理完成: shapefile_M
开始处理图层: shapefile_S
图层处理完成: shapefile_S
开始处理图层: shapefile_G
图层处理完成: shapefile_G


In [ ]:
# 列出所有待处理的 POI 图层名称及其对应的图层对象
poi_layers = {
    "shapefile_R": r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project",
    "shapefile_A": r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project",
    "shapefile_B": r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project",
    "shapefile_M": r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project",
    "shapefile_S": r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project",
    "shapefile_G": r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project"
}

# 对每个图层执行处理，并加上进度说明
for poi_layer_name, poi_layer in poi_layers.items():
    process_poi_layer(poi_layer_name, poi_layer)

#### 其他

In [ ]:
# 为POI图层中的每个点构建四叉树并分配格网ID
build_quadtree_and_assign_grid_ids(poi_layer, root)

In [ ]:
# 计算每个格网内POI的数量
grid_poi_counts = calculate_grid_poi_counts(poi_layer)

In [ ]:
# 为每个POI点分配权重
assign_weights_to_pois(poi_layer, grid_poi_counts)